## [네이버 프로야구 데이터 수집](https://sports.news.naver.com/kbaseball/schedule/index#)
### 2022.01.29

- 소스보기에 데이터가 없다. -> ajax, iframe, 웹소켓
- 우클릭 메뉴에 프레임 소스보기가 없다. -> ajax, 웹소켓
- 크롬 개발자 도구의 network 탭에 xhr로 표시된다 -> ajax
- ajax 통신 주소를 통해 requets로 요청하고, bs4로 분석

In [1]:
# 수집한 데이터 저장용
import numpy as np
import pandas as pd
# 요청
import requests
# 수집
import bs4
# 기타
import time
import os
from IPython.display import clear_output

In [2]:
# 요청함수
def getSource(site) :
    # 헤더 정보 셋팅
    header_info = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
    }
    
    # 요청한다
    response = requests.get(site, headers=header_info)
    return response.text.strip()

In [3]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source, fileName) :
    
    # 데이터를 담을 딕셔너리
    data_dict = {
        '경기날짜' : [],
        '팀1이름' : [],
        '팀2이름' : [],
        '팀1점수' : [],
        '팀2점수' : []
    }
    
    # 경기날짜
    # 오늘 날짜에 해당하는 태그를 가져온다.
    a1 = source.select_one('body > div > ul > li.on > a')
#     print(a1)
    a2 = a1.attrs['onclick']
#     print(a2)
    data1 = a2.split("'")
#     print(data1[1])

    # 경기가 정상적으로 종료된 경기만 가져온다.
    li_list = source.select('#todaySchedule > li.end')
    
    for li_tag in li_list :
        # print(li_tag)
        # print('--------------------')
        
        # 경기날짜
        data_dict['경기날짜'].append(data1[1])
        
        # 팀1이름
        a1 = li_tag.select_one('div.vs_lft > p > strong')
        if a1 == None :
            data_dict['팀1이름'].append(np.nan)
        else :
            data10 = a1.text.strip()
            data_dict['팀1이름'].append(data10)
            
        # 팀2이름
        a1 = li_tag.select_one('div.vs_rgt > p > strong')
        if a1 == None :
            data_dict['팀2이름'].append(np.nan)
        else :
            data10 = a1.text.strip()
            data_dict['팀2이름'].append(data10)
            
        # 팀1점수
        a1 = li_tag.select_one('div.vs_lft > strong')
        if a1 == None :
            data_dict['팀1점수'].append(np.nan)
        else :
            data10 = a1.text.strip()
            data_dict['팀1점수'].append(data10)
        
        # 팀2점수
        a1 = li_tag.select_one('div.vs_rgt > strong')
        if a1 == None :
            data_dict['팀2점수'].append(np.nan)
        else :
            data10 = a1.text.strip()
            data_dict['팀2점수'].append(data10)
        
#     print(data_dict)

    # 데이터 프레임을 생성한다.
    df1 = pd.DataFrame(data_dict)
    # display(df1)
    
    # 저장한다.
    # 파일이 없다면 : 첫 번째 저장
    if os.path.exists(fileName) == False :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False)
    # 파일이 있다면..
    else :
        df1.to_csv(fileName, encoding='utf-8-sig', index=False, mode='a', header=None)

In [4]:
def getNext(source) :
    # 내일 날짜에 해당하는 태그를 가져온다.
    # 오늘 날짜 태그의 다음 형제 태그(li.on + li)를 접근한다.
    a1 = source.select_one('body > div > ul > li.on + li > a') # +  li => 바로 다음 형제 태그
    # print(a1)
    a2 = a1.attrs['onclick']
    # print(a2)
    data1 = a2.split("'")
    
    # print(data1[1])
    return data1[1]

In [5]:
# 수집 날짜
today = '20080329'
fileName = 'kbo_info.csv'

# 파일이 존재하면 삭제한다.
if os.path.exists(fileName) == True :
    os.remove(fileName)

while True: 
    # 딜레이
    time.sleep(1)
    
    # 출력 청소
    clear_output(wait=True)
    
    print(f'{today} 수집중...')
    
    # 요청 주소
    site = f'https://sports.news.naver.com/schedule/scoreBoard?date={today}&category=kbo'

    # 요청한다.
    a1 = getSource(site)
    source = bs4.BeautifulSoup(a1)

    # 데이터를 수집한다
    getData(source, fileName)

    # 다음 날짜를 받는다.
    today = getNext(source)
    
    # 2022년이 되면 중단시킨다.
    if today.startswith('2022') :
        break

print('수집완료')

20080415 수집중...


KeyboardInterrupt: 